<a href="https://colab.research.google.com/github/UFM-Market-Trends/UFM-Market-Trends-SDK/blob/main/examples/6_Sistema_bancario_consulta_33_SIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install pyxlsb
!pip install fpdf
import datetime
!pip install git+https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git
import ufmtrends_sdk as trends
today = datetime.date.today().strftime("%Y-%m-%d")
import requests
!pip install lxml

  Using cached pyxlsb-1.0.10-py2.py3-none-any.whl (23 kB)
  Using cached fpdf-1.7.2-py2.py3-none-any.whl
  Cloning https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git to /tmp/pip-req-build-nhb1ow_4
  Running command git clone --filter=blob:none --quiet https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git /tmp/pip-req-build-nhb1ow_4
  Resolved https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git to commit bc267875f263adc24695f4afc8df549f6678ee0b
  Preparing metadata (setup.py) ... done
  Created wheel for ufmtrends-sdk: filename=ufmtrends_sdk-0.1.1-py3-none-any.whl size=13005 sha256=c6035fb14fe0e47bb8a434297a4a0597b9b4fe131c7611aa6c1b423a8e9d8846
  Stored in directory: /tmp/pip-ephem-wheel-cache-uf2fsuba/wheels/d8/63/72/fd94fcb7285b31cd0125721e8d1340b8842b4efc3b6d212027
Successfully built ufmtrends-sdk
  Using cached lxml-4.9.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.6 MB)


In [2]:
# # https://colab.research.google.com/notebooks/snippets/gcs.ipynb
# from google.colab import auth
# auth.authenticate_user()

# project_id = 'ufm-market-trends'
# !gcloud config set project {project_id}

# Sistema bancario - consulta 33 SIB

In [3]:
# Agricultura, ganadería, silvicultura y pesca
AGRICULTURE = "Agricultura"
# Explotación de minas y canteras
MINES = "Minas y canteras"
# Industrias manufactureras
MANUFACTURE = "Manufactura"
# Suministro de electricidad, agua y saneamiento
WATER_AND_ELECTRICITY = "Agua y electricidad"
# Construcción
CONSTRUCTION = "Construcción"
# Comercio y reparación de vehículos
COMMERCE = "Comercio"
# Transporte y almacenamiento
TRANSPORT = "Transporte y almacenamiento"
# Actividades de alojamiento y servicio de comidas
HOTELS_AND_RESTAURANTS = "Hoteles y restaurantes"
# Información y comunicaciones
COMMUNICATIONS = "Información y comunicaciones"
# Actividades financieras y de seguros
FINANCE = "Actividades financieras y de seguros"
# Actividades inmobiliarias
REAL_STATE = "Actividades inmobiliarias"
# Actividades profesionales, científicas y técnicas
PROFESSIONAL_ACTIVITIES = "Actividades profesionales"
# Actividades de servicios administrativos y de apoyo
ADMINISTRATION_SERVICES = "Administración y servicios"
# Administración pública y defensa
PUBLIC_ADMINISTRATION = "Sector público"
# Enseñanza
EDUCATION = "Educación"
# Salud
HEALTH_CARE = "Salud"
# Otras actividades de servicios
OTHER_ACTIVITIES = "Otras actividades"

# Impuestos netos de subvenciones a los productos
NET_SUBSIDY_TAXES = "Impuestos de subvenciones"

In [4]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pct_change.html

def download_credit_sectors(credit_data_url, year, month):
  '''
  '''  
  '''
  El .xls, al descargarlo y aplicar pd.read_html, retorna 
  varios dataframes. Nos interesa el correspondiente a "plazo
  original" por bancos nacionales. Por la estructura del .xls, corresponde
  al index 3 en la lista.

  Nos interesa 
  SALDOS DE PRÉSTAMOS Y DESCUENTOS POR PLAZO ORIGINAL BANCOS NACIONALES
  '''
  # username = 'josealvarez97'
  # (mecesario mientras repo sea privado...)
  # token = '5d806739ecee103551b230ab5d4cbb60df86174a' 
  # github_session = requests.Session()

  download = requests.get(credit_data_url).content
  credit_sectors_dataframe = pd.read_html(download)[3]
  # print(pd.read_html(download)[3]) # Plazo original

  def col_to_float(df_col):
    return df_col.astype('string').astype('float')
  # print(credit_sectors_dataframe.iloc[1:,1:].apply(
  #     col_to_float, axis=1).sum(axis=1))
  credit_sectors_dataframe['sum'] = credit_sectors_dataframe.iloc[1:,1:].apply(
      col_to_float, axis=1).sum(axis=1)
  # credit_sectors_dataframe = credit_sectors_dataframe.iloc[:,:2] 
  credit_sectors_dataframe = credit_sectors_dataframe.iloc[:,[0,-1]] 
  credit_sectors_dataframe = credit_sectors_dataframe.transpose()

  new_header = credit_sectors_dataframe.iloc[0]
  credit_sectors_dataframe = credit_sectors_dataframe[1:] 
  credit_sectors_dataframe.columns = new_header
  
  '''
  Por la estructura del .xls (puedes consultar la página de SIB para convencerte),
  hay muchas columnas en el dataframe que son subcolumnas de anteriores. Por tanto,
  las vamos a eliminar. No las necesitamos.
  '''
  credit_sectors_dataframe = credit_sectors_dataframe.iloc[:,[1,2,7,8,9,10,12,13,14,15,16]]

  '''
  Los sectores son casi los mismos que encontramos en el crecimiento de PIB por sector.
  Por tanto, reusaremos las constantes que declaramos para definir sus nombres,
  excepto por unas pocas excepciones.

  (Algunos nombres son simplificados, como agricultura).
  '''

  CONSUMPTION_AND_TRANSFERENCES = "Consumo"
  FINANCE_AND_REAL_STATE = "Finanzas e inmuebles"
  COMMUNAL_SERVICES = "Servicios comunales"
  # print(credit_sectors_dataframe.columns)
  credit_sectors_dataframe.columns = [CONSUMPTION_AND_TRANSFERENCES, AGRICULTURE,
                                      MINES, MANUFACTURE, WATER_AND_ELECTRICITY,
                                      CONSTRUCTION, COMMERCE, TRANSPORT, 
                                      FINANCE_AND_REAL_STATE, COMMUNAL_SERVICES,
                                      'total']
  # print(credit_sectors_dataframe.columns)
  date = datetime.datetime(year=year, month=month, day=1)
  credit_sectors_dataframe.insert(loc=0, column='date', value=[date])
  credit_sectors_dataframe.insert(loc=1, column='month', 
                                  value=[f'{date.year}-{date.month}'])

 
  credit_sectors_dataframe.iloc[:,2:] = credit_sectors_dataframe.iloc[:,2:].apply(
      col_to_float, axis=1)

  return credit_sectors_dataframe


In [5]:
# this is just an example, that is why it's using an old date
test_url =  "https://storage.googleapis.com/ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion/descarga-sib-consulta-33/2019-09-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls"
download_credit_sectors(test_url,2019,9)

,date,month,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
sum,2019-09-01,2019-9,62224863.0,8726743.0,578845.0,20843948.0,14199251.0,12327213.0,23100251.0,1744229.0,19468261.0,6715354.0,169928956.0


In [6]:

# initial_credit_data_df = download_credit_sectors(initial_credit_data_url, 2020, 11)
# final_credit_data_df = download_credit_sectors(final_credit_data_url, 2021, 11)

# credit_sectors_df = initial_credit_data_df.append(final_credit_data_df)
# credit_sectors_df.iloc[:,2:] = credit_sectors_df.iloc[:,2:].pct_change()
# credit_sectors_df


In [7]:
start_year = 2013
start_month = 11

today_datetime = datetime.date.today()
total_months = (today_datetime.year - start_year) * 12 + today_datetime.month

credit_sectors_df = None
first_iteration = True

for month in range(start_month-1,total_months):
  year_i = start_year + (month//12)  
  month_i = month % 12 + 1
  
  xls_filename = f'{year_i}-{str(month_i).zfill(2)}-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls'
  url_base_address = "https://storage.googleapis.com/ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion/descarga-sib-consulta-33/"
  url_address = url_base_address + xls_filename
  print(xls_filename)

  if first_iteration:
    credit_sectors_df = download_credit_sectors(url_address, year_i, month_i)
    first_iteration = False
  elif requests.get(url_address).status_code == 404:
    print(f"Breaking at {xls_filename} (not available yet)")
    break
  else:
      credit_sectors_df = credit_sectors_df.append(download_credit_sectors(url_address, year_i, month_i))

credit_sectors_df = credit_sectors_df.reset_index(drop=True)
credit_sectors_df

2013-11-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2013-12-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-01-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-02-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-03-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-04-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-05-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-06-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-07-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-08-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-09-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-10-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-11-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2014-12-Saldos-de-Prestamos-por-Estratificacion-SIB-Consulta_33.xls
2015-01-Saldos-de-Prestamos-por-Estratificacion-

,date,month,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
0,2013-11-01,2013-11,36892916.0,6835074.0,137744.0,15408772.0,7394735.0,9306607.0,18571268.0,1782861.0,12273527.0,4368914.0,112972417.0
1,2013-12-01,2013-12,37329510.0,6929748.0,127441.0,14569868.0,7430696.0,9321386.0,18293989.0,1779141.0,12485997.0,4359313.0,112627086.0
2,2014-01-01,2014-1,37577216.0,7076663.0,129519.0,14614357.0,7478307.0,9312299.0,18172189.0,1781409.0,12216709.0,4365435.0,112724104.0
3,2014-02-01,2014-2,37924691.0,7007784.0,157783.0,14735520.0,7401901.0,9271314.0,17989030.0,1772965.0,12136203.0,3737973.0,112135162.0
4,2014-03-01,2014-3,38326895.0,7057886.0,157233.0,15100803.0,7496987.0,9256045.0,18273096.0,1777106.0,12051414.0,3718065.0,113215531.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2022-07-01,2022-7,84239228.0,9410801.0,293298.0,29896257.0,15687462.0,14151288.0,31195966.0,2383985.0,28178044.0,9706136.0,225142462.0
105,2022-08-01,2022-8,85681140.0,9203417.0,304061.0,30781997.0,15418140.0,14308307.0,30844110.0,2458200.0,28268363.0,9743910.0,227011645.0
106,2022-09-01,2022-9,87229982.0,9318603.0,209569.0,32317382.0,15477948.0,14395420.0,31276206.0,2600449.0,29474043.0,9629633.0,231929234.0
107,2022-10-01,2022-10,88241628.0,9429795.0,194747.0,33144898.0,15235258.0,14579181.0,32098211.0,2609627.0,29945656.0,9357393.0,234836392.0


In [8]:
credit_sectors_csv = "Credito-Por-Sector-Mensual-Miles-Quetzales.csv"
credit_sectors_df.to_csv(credit_sectors_csv, index=False)

In [9]:
bucket_path = 'ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion'
csv_file = credit_sectors_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}

Copying file://Credito-Por-Sector-Mensual-Miles-Quetzales.csv [Content-Type=text/csv]...
/ [1 files][ 14.5 KiB/ 14.5 KiB]                                                
Operation completed over 1 objects/14.5 KiB.                                     
Copying gs://ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion/Credito-Por-Sector-Mensual-Miles-Quetzales.csv [Content-Type=text/csv]...
/ [1 files][ 14.5 KiB/ 14.5 KiB]                                                
Operation completed over 1 objects/14.5 KiB.                                     
date,month,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
2013-11-01,2013-11,36892916.0,6835074.0,137744.0,15408772.0,7394735.0,9306607.0,18571268.0,1782861.0,12273527.0,4368914.0,112972417.0
2013-12-01,2013-12,37329510.0,6929748.0,127441.0,14569868.0,7430696.0,9321386.0,1829

# Crecimiento credito por sector

acum 12 meses

In [10]:
credit_sectors_accum_growth_df = credit_sectors_df
credit_sectors_accum_growth_df.iloc[:,2:] = credit_sectors_accum_growth_df.iloc[:,2:].transform(lambda x: trends.get_accumulated_variation(x,lot_size=12))
credit_sectors_accum_growth_df

,date,month,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
0,2013-11-01,2013-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-12-01,2013-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-01,2014-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-02-01,2014-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-03-01,2014-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2022-07-01,2022-7,0.155164,0.050794,-0.560960,0.123232,0.131589,0.010952,0.151401,0.123945,0.158972,0.146687,0.130401
105,2022-08-01,2022-8,0.159467,0.048886,-0.589978,0.145587,0.147464,0.001816,0.155590,0.119159,0.147998,0.142977,0.134068
106,2022-09-01,2022-9,0.163618,0.049734,-0.635361,0.169400,0.166726,-0.002208,0.154442,0.118581,0.148371,0.139303,0.139329
107,2022-10-01,2022-10,0.166595,0.042267,-0.607683,0.192439,0.160962,-0.003349,0.158630,0.117801,0.149726,0.140641,0.143583


In [11]:
credit_sectors_accum_growth_csv = "Crecimiento-Credito-Por-Sector-Acum-12-Meses.csv"
credit_sectors_accum_growth_df.to_csv(credit_sectors_accum_growth_csv, index=False)

In [12]:
bucket_path = 'ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion'
csv_file = credit_sectors_accum_growth_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}

Copying file://Crecimiento-Credito-Por-Sector-Acum-12-Meses.csv [Content-Type=text/csv]...
/ [1 files][ 21.1 KiB/ 21.1 KiB]                                                
Operation completed over 1 objects/21.1 KiB.                                     
Copying gs://ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion/Crecimiento-Credito-Por-Sector-Acum-12-Meses.csv [Content-Type=text/csv]...
/ [1 files][ 21.1 KiB/ 21.1 KiB]                                                
Operation completed over 1 objects/21.1 KiB.                                     
date,month,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
2013-11-01,2013-11,,,,,,,,,,,
2013-12-01,2013-12,,,,,,,,,,,
2014-01-01,2014-1,,,,,,,,,,,
2014-02-01,2014-2,,,,,,,,,,,
2014-03-01,2014-3,,,,,,,,,,,
2014-04-01,2014-4,,,,,,,,,,,
2014-05-01,2014-5,,,,,,,,,,,
2014-06-0

In [13]:
latest_credit_sectors_accum_growth_csv = "Crecimiento-Credito-Por-Sector-Acum-12-Meses-Ultimo-Mes.csv"
credit_sectors_accum_growth_df.iloc[-1:,2:].to_csv(latest_credit_sectors_accum_growth_csv, index=False)
credit_sectors_accum_growth_df.iloc[-1:,2:]

,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
108,0.168562,0.039617,-0.577096,0.209882,0.143579,-0.002629,0.16132,0.117472,0.144822,0.139718,0.145277


In [14]:
bucket_path = 'ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion'
csv_file = latest_credit_sectors_accum_growth_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}

Copying file://Crecimiento-Credito-Por-Sector-Acum-12-Meses-Ultimo-Mes.csv [Content-Type=text/csv]...
/ [1 files][  388.0 B/  388.0 B]                                                
Operation completed over 1 objects/388.0 B.                                      
Copying gs://ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion/Crecimiento-Credito-Por-Sector-Acum-12-Meses-Ultimo-Mes.csv [Content-Type=text/csv]...
/ [1 files][  388.0 B/  388.0 B]                                                
Operation completed over 1 objects/388.0 B.                                      
Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
0.16856192891863753,0.039617100550120396,-0.5770960583223226,0.20988202054107902,0.1435785191709367,-0.0026294374031533277,0.1613196894336734,0.1174715759796625,0.14482157886167318,0.13971751006627173,0.

interanual

In [15]:
credit_sectors_growth_df = credit_sectors_df
credit_sectors_growth_df.iloc[:,2:] = credit_sectors_growth_df.iloc[:,2:].transform(
    lambda x: trends.get_yearonyear_variation(list(x),inter_values_size=12))
credit_sectors_growth_df

,date,month,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
0,2013-11-01,2013-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-12-01,2013-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-01,2014-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-02-01,2014-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-03-01,2014-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2022-07-01,2022-7,1.162596,4.961820,-14.326682,1.401004,-4.377856,-0.899164,2.541717,0.049967,0.101435,-0.302787,0.852038
105,2022-08-01,2022-8,1.016389,3.656788,116.390027,2.603010,-5.686827,-0.982014,2.422544,-0.037219,-0.037196,-0.337002,0.823704
106,2022-09-01,2022-9,0.892172,2.765885,36.132228,3.776965,-7.288859,-1.024584,1.621315,-0.086690,-0.014845,-0.388600,0.800980
107,2022-10-01,2022-10,0.759098,0.837413,6.504468,4.993336,-14.265701,-1.043193,1.400457,-0.122134,0.018856,-0.329559,0.785263


In [16]:
credit_sectors_growth_csv = "Crecimiento-Interanual-Credito-Por-Sector-Miles-Quetzales.csv"
credit_sectors_growth_df.to_csv(credit_sectors_growth_csv, index=False)

In [17]:
bucket_path = 'ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion'
csv_file = credit_sectors_growth_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}

Copying file://Crecimiento-Interanual-Credito-Por-Sector-Miles-Quetzales.csv [Content-Type=text/csv]...
/ [1 files][ 18.0 KiB/ 18.0 KiB]                                                
Operation completed over 1 objects/18.0 KiB.                                     
Copying gs://ufm-market-trends/guatemala/sistema-bancario/super-intendencia-bancos/saldos-prestamos-estratificacion/Crecimiento-Interanual-Credito-Por-Sector-Miles-Quetzales.csv [Content-Type=text/csv]...
/ [1 files][ 18.0 KiB/ 18.0 KiB]                                                
Operation completed over 1 objects/18.0 KiB.                                     
date,month,Consumo,Agricultura,Minas y canteras,Manufactura,Agua y electricidad,Construcción,Comercio,Transporte y almacenamiento,Finanzas e inmuebles,Servicios comunales,total
2013-11-01,2013-11,,,,,,,,,,,
2013-12-01,2013-12,,,,,,,,,,,
2014-01-01,2014-1,,,,,,,,,,,
2014-02-01,2014-2,,,,,,,,,,,
2014-03-01,2014-3,,,,,,,,,,,
2014-04-01,2014-4,,,,,,,,,,,
2014-05-01,2